In [3]:
import os 
import numpy as np
import pandas as pd
from utils.utils import *
import nibabel as nib

In [4]:
model_path = os.path.join(fits_dir, 'fits_inversed', 'subj01', 'fits_subj01_train_V1_inversed.npy')
model = np.load(model_path, allow_pickle=True)
model.shape

(33797, 9)

In [5]:
"""
I dont think this is correct: because I take values from when we're SAMPLING in V1, not when we're looking at V1's voxel
It should be the other way around; which means loading all the fits and then averaging based on mask's indices
"""


columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang"] #, "roi", "fit_with"]  # replace fit with with the best explain? just one value 
results = -np.ones((2, len(subj_list), len(rois.keys()), len(rois.keys())))

print(results.shape)


for i, sub in enumerate(subj_list):
    if sub == 'subj06' or sub == 'subj08':
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.nans.{sub}.testrois.npy')
    else:
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.{sub}.testrois.npy')
            # find cutoff for lh and rh division
    if sub == 'subj06' or sub == 'subj08':
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.{sub}.nans.testrois.mgz')
        mask = nib.load(mask_cutoff_file).get_fdata().squeeze()
        mask = mask[mask != 0]
        cutoff = mask.shape
    else: 
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.short.reduced.{sub}.testrois.npy')
        mask = np.load(mask_cutoff_file)
        cutoff = mask.shape
    maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)
    n_voxels = maskdata.shape[0]
    belongs = []
    for k in maskdata:
        belongs.append(sior[k])

    belongs = np.array(belongs)
    for j, roi in enumerate(rois.keys()):
        fits_path = os.path.join(fits_dir, 'fits_inversed', sub, f'fits_{sub}_train_{roi}_inversed.npy')

        fits_matrix = np.load(fits_path, allow_pickle=True)
       
        scores_per_roi_lh = np.array([np.mean(fits_matrix[:cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
        scores_per_roi_rh = np.array([np.mean(fits_matrix[cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

        results[0, i, j] = scores_per_roi_lh
        results[1, i, j] = scores_per_roi_rh

results[:, 0]
        
        



(2, 8, 12, 12)


array([[[ 3.32431260e-02,  2.62957198e-02,  2.17434852e-02,
          1.20587117e-02,  8.76861622e-03,  8.56351291e-03,
         -3.67543457e-03, -1.10208988e-03, -3.66743731e-04,
         -2.47612027e-03, -1.82840276e-03, -2.80433248e-04],
        [ 2.55351658e-02,  2.21965043e-02,  2.06439289e-02,
          6.24784098e-03,  3.52547123e-03, -3.03241147e-04,
         -4.40239402e-03, -1.69117053e-03,  4.25234715e-03,
         -1.17467085e-03,  4.02479518e-04,  6.08951347e-04],
        [ 2.20140192e-02,  2.59257829e-02,  2.71559116e-02,
          1.82983251e-02,  7.54301454e-03,  8.98253225e-03,
         -2.11239159e-03, -7.41370614e-04,  4.50432202e-03,
         -6.70168814e-04, -2.55327957e-03, -7.47172012e-04],
        [ 2.26664169e-02,  2.79696826e-02,  2.95757549e-02,
          1.19209325e-02,  6.61870452e-03, -6.75151223e-04,
         -3.45920152e-03, -8.94703468e-04,  1.13014056e-02,
          2.81889158e-04,  1.06072891e-03,  1.70517097e-03],
        [ 1.89997692e-02,  2.7532850

In [6]:
subj01_results_lh = pd.DataFrame(results[0, 0], columns=rois.keys(), index=rois.keys())
subj01_results_lh["best_roi"]  = subj01_results_lh.idxmax(axis=1)
subj01_results_lh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.033243,0.026296,0.021743,0.012059,0.008769,0.008564,-0.003675,-0.001102,-0.000367,-0.002476,-0.001828,-0.000280,V1
V2,0.025535,0.022197,0.020644,0.006248,0.003525,-0.000303,-0.004402,-0.001691,0.004252,-0.001175,0.000402,0.000609,V1
V3,0.022014,0.025926,0.027156,0.018298,0.007543,0.008983,-0.002112,-0.000741,0.004504,-0.000670,-0.002553,-0.000747,V3
hV4,0.022666,0.027970,0.029576,0.011921,0.006619,-0.000675,-0.003459,-0.000895,0.011301,0.000282,0.001061,0.001705,V3
VO-1,0.019000,0.027533,0.032470,0.020624,0.010306,0.012846,-0.000152,0.000987,0.006260,0.001056,-0.002205,0.000058,V3
VO-2,0.029126,0.035674,0.042959,0.019990,0.011851,0.004781,-0.001541,0.000793,0.017687,0.003657,0.001837,0.002664,V3
PHC-1,0.004986,0.011608,0.017759,0.016739,0.011089,0.009227,0.000299,0.001138,0.003811,0.002090,-0.000376,0.000478,V3
PHC-2,0.006003,0.010119,0.013326,0.010516,0.045346,0.038280,0.034491,0.042961,0.010487,0.033561,0.071529,0.063265,TO-1
LO-1,0.007554,0.010915,0.012100,0.009089,0.029002,0.042337,0.017788,0.023905,0.005483,0.017984,0.038219,0.032226,VO-2
LO-2,0.000999,0.002009,0.003855,0.003690,0.023794,0.025387,0.035481,0.041008,0.012282,0.027209,0.059138,0.058223,TO-1


In [7]:
subj01_results_rh = pd.DataFrame(results[1, 0], columns=rois.keys(), index=rois.keys())
subj01_results_rh["best_roi"]  = subj01_results_rh.idxmax(axis=1)
subj01_results_rh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.006159,0.013197,0.011803,0.006188,-0.004491,-0.008362,-0.007092,-0.006327,0.000516,-0.004508,-0.000158,-0.000376,V2
V2,0.030054,0.026462,0.023849,0.001388,-0.002515,-0.006414,-0.006345,-0.004847,0.010666,-0.000587,0.000542,0.000373,V1
V3,0.019979,0.035076,0.029437,0.017708,-0.002298,-0.005372,-0.004818,-0.004145,0.011774,0.003130,0.002348,-0.000654,V2
hV4,0.023744,0.034524,0.026778,0.002219,-0.002451,-0.006956,-0.006235,-0.004054,0.024719,0.004418,0.005563,0.002178,V2
VO-1,0.016419,0.028426,0.031096,0.020774,-0.000293,-0.003584,-0.002483,-0.002077,0.016607,0.005119,0.003762,-0.000464,V3
VO-2,0.021211,0.030611,0.031972,0.004618,0.000096,-0.005986,-0.005880,-0.003701,0.031237,0.008778,0.009355,0.003846,V3
PHC-1,0.005832,0.018630,0.022910,0.040554,0.006223,-0.001228,-0.001446,-0.001999,0.011449,0.002021,0.012251,0.003487,hV4
PHC-2,0.002161,0.005589,0.008048,0.008090,0.016420,0.036903,0.048406,0.049857,0.006419,0.005833,0.111117,0.062498,TO-1
LO-1,0.003026,0.004592,0.005773,0.001612,0.009269,0.027603,0.036078,0.047247,0.003043,0.001315,0.052409,0.027868,TO-1
LO-2,0.001889,0.004418,0.008331,0.003766,0.021680,0.040176,0.068090,0.077517,0.007792,0.010345,0.095046,0.065087,TO-1


In [8]:
results.shape

(2, 8, 12, 12)

In [9]:
results[0, 0, 0]

array([ 0.03324313,  0.02629572,  0.02174349,  0.01205871,  0.00876862,
        0.00856351, -0.00367543, -0.00110209, -0.00036674, -0.00247612,
       -0.0018284 , -0.00028043])

In [16]:
"""
I dont think this is correct: because I take values from when we're SAMPLING in V1, not when we're looking at V1's voxel
It should be the other way around; which means loading all the fits and then averaging based on mask's indices
"""

columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang"] #, "roi", "fit_with"]  # replace fit with with the best explain? just one value 
results = -np.ones((2, len(subj_list), len(rois.keys()), len(rois.keys())))

print(results.shape)


for i, sub in enumerate(subj_list):
    if sub == 'subj06' or sub == 'subj08':
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.nans.{sub}.testrois.npy')
    else:
        maskdata_file = os.path.join(mask_dir, sub, f'short.reduced.{sub}.testrois.npy')
            # find cutoff for lh and rh division
    if sub == 'subj06' or sub == 'subj08':
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.{sub}.nans.testrois.mgz')
        mask = nib.load(mask_cutoff_file).get_fdata().squeeze()
        mask = mask[mask != 0]
        cutoff = mask.shape
    else: 
        mask_cutoff_file = os.path.join(mask_dir, sub, f'lh.short.reduced.{sub}.testrois.npy')
        mask = np.load(mask_cutoff_file)
        cutoff = mask.shape
    maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)
    n_voxels = maskdata.shape[0]
    belongs = []
    for k in maskdata:
        belongs.append(sior[k])

    belongs = np.array(belongs)

    fits_subj = np.zeros((len(rois.keys()), n_voxels, len(columns)))
    for j, roi in enumerate(rois.keys()):

        



        
        fits_path = os.path.join(fits_dir, 'fits_inversed', sub, f'fits_{sub}_train_{roi}_inversed.npy')

        fits_matrix = np.load(fits_path, allow_pickle=True)
        
        fits_subj[j] = fits_matrix
       
       # scores_per_roi_lh = np.array([np.mean(fits_matrix[:cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
       # scores_per_roi_rh = np.array([np.mean(fits_matrix[cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

      #  results[0, i, j] = scores_per_roi_lh
       # results[1, i, j] = scores_per_roi_rh\
    print(np.mean(fits_subj[0, :cutoff[0],5][belongs[:cutoff[0]] == 'V1']))
    for k, roi in enumerate(rois.keys()):
        scores_per_roi_lh = np.array([np.mean(fits_subj[k, :cutoff[0], 5][belongs[:cutoff[0]] == target_roi]) for target_roi in rois.keys()])
        scores_per_roi_rh = np.array([np.mean(fits_subj[k, cutoff[0]:, 5][belongs[cutoff[0]:] == target_roi]) for target_roi in rois.keys()])

        results[0, i, k] = scores_per_roi_lh
        results[1, i, k] = scores_per_roi_rh
    


(2, 8, 12, 12)
0.033243126006174385
0.03127391114635417
-0.005450302514527701
0.0154336386515519
0.01522661283911032
0.00939469784049114
-0.00072955255971458
0.01313440257321492


In [17]:
subj01_results_lh = pd.DataFrame(results[0, 0], columns=rois.keys(), index=rois.keys())
subj01_results_lh["best_roi"]  = subj01_results_lh.idxmax(axis=1)
subj01_results_lh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.033243,0.026296,0.021743,0.012059,0.008769,0.008564,-0.003675,-0.001102,-0.000367,-0.002476,-0.001828,-0.000280,V1
V2,0.025535,0.022197,0.020644,0.006248,0.003525,-0.000303,-0.004402,-0.001691,0.004252,-0.001175,0.000402,0.000609,V1
V3,0.022014,0.025926,0.027156,0.018298,0.007543,0.008983,-0.002112,-0.000741,0.004504,-0.000670,-0.002553,-0.000747,V3
hV4,0.022666,0.027970,0.029576,0.011921,0.006619,-0.000675,-0.003459,-0.000895,0.011301,0.000282,0.001061,0.001705,V3
VO-1,0.019000,0.027533,0.032470,0.020624,0.010306,0.012846,-0.000152,0.000987,0.006260,0.001056,-0.002205,0.000058,V3
VO-2,0.029126,0.035674,0.042959,0.019990,0.011851,0.004781,-0.001541,0.000793,0.017687,0.003657,0.001837,0.002664,V3
PHC-1,0.004986,0.011608,0.017759,0.016739,0.011089,0.009227,0.000299,0.001138,0.003811,0.002090,-0.000376,0.000478,V3
PHC-2,0.006003,0.010119,0.013326,0.010516,0.045346,0.038280,0.034491,0.042961,0.010487,0.033561,0.071529,0.063265,TO-1
LO-1,0.007554,0.010915,0.012100,0.009089,0.029002,0.042337,0.017788,0.023905,0.005483,0.017984,0.038219,0.032226,VO-2
LO-2,0.000999,0.002009,0.003855,0.003690,0.023794,0.025387,0.035481,0.041008,0.012282,0.027209,0.059138,0.058223,TO-1


In [18]:
subj01_results_rh = pd.DataFrame(results[1, 0], columns=rois.keys(), index=rois.keys())
subj01_results_rh["best_roi"]  = subj01_results_rh.idxmax(axis=1)
subj01_results_rh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2,best_roi
V1,0.006159,0.013197,0.011803,0.006188,-0.004491,-0.008362,-0.007092,-0.006327,0.000516,-0.004508,-0.000158,-0.000376,V2
V2,0.030054,0.026462,0.023849,0.001388,-0.002515,-0.006414,-0.006345,-0.004847,0.010666,-0.000587,0.000542,0.000373,V1
V3,0.019979,0.035076,0.029437,0.017708,-0.002298,-0.005372,-0.004818,-0.004145,0.011774,0.003130,0.002348,-0.000654,V2
hV4,0.023744,0.034524,0.026778,0.002219,-0.002451,-0.006956,-0.006235,-0.004054,0.024719,0.004418,0.005563,0.002178,V2
VO-1,0.016419,0.028426,0.031096,0.020774,-0.000293,-0.003584,-0.002483,-0.002077,0.016607,0.005119,0.003762,-0.000464,V3
VO-2,0.021211,0.030611,0.031972,0.004618,0.000096,-0.005986,-0.005880,-0.003701,0.031237,0.008778,0.009355,0.003846,V3
PHC-1,0.005832,0.018630,0.022910,0.040554,0.006223,-0.001228,-0.001446,-0.001999,0.011449,0.002021,0.012251,0.003487,hV4
PHC-2,0.002161,0.005589,0.008048,0.008090,0.016420,0.036903,0.048406,0.049857,0.006419,0.005833,0.111117,0.062498,TO-1
LO-1,0.003026,0.004592,0.005773,0.001612,0.009269,0.027603,0.036078,0.047247,0.003043,0.001315,0.052409,0.027868,TO-1
LO-2,0.001889,0.004418,0.008331,0.003766,0.021680,0.040176,0.068090,0.077517,0.007792,0.010345,0.095046,0.065087,TO-1
